In [11]:
%%capture
import time
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input, State, ALL, ALLSMALLER, MATCH
from dash.exceptions import PreventUpdate
import pandas as pd

poverty = pd.read_csv('../../data/poverty.csv', low_memory=False)
countries = poverty[poverty['is_country']]['Country Name'].drop_duplicates().sort_values()

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Br(),
    html.Div(id='feedback'),
    dbc.Label("Create your own dropdown, add options one per line:"),
    dbc.Textarea(id='text', cols=40, rows=5),
    html.Br(),
    dbc.Button("Set options", id='button'),
    html.Br(),
    dcc.Dropdown(id='dropdown'),
    dcc.Graph(id='chart')
])


@app.callback(Output('dropdown', 'options'),
              Output('feedback', 'children'),
              Input('button', 'n_clicks'),
              State('text', 'value'))
def set_dropdown_options(n_clicks, options):
    if not n_clicks:
        raise PreventUpdate
    text = options.split()
    message = dbc.Alert(f"Success! you added the options: {', '.join(text)}", 
                        color='success',
                        dismissable=True)
    options = [{'label': t, 'value': t} for t in text]
    return options, message

@app.callback(Output('chart', 'figure'),
              Input('dropdown', 'value'))
def create_population_chart(country_code):
    if not country_code:
        raise PreventUpdate
    df = poverty[poverty['Country Code']==country_code]
    return px.line(df, x='year', y='Population, total', title=f"Population of {country_code}")

app.run_server(host='0.0.0.0',debug= True, height=1500, port=3000)

In [ ]:
! echo 'Your app is running on: '$EDUCATIVE_LIVE_VM_URL':3000'